In [95]:
import pandas as pd
import dash
from dash import html, dash_table, dcc, Dash
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
# from src.components.database_connection import load_division_goods_monthwise_outward_data
import datetime


import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Load environment variables from .env file
load_dotenv()

# Access the database connection details
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Define the database connection URI
DB_URI = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create a SQLAlchemy engine
engine = create_engine(DB_URI)

conn = engine.connect()

def load_division_good_yearly_depowisesoutward_data(selected_division):
    # Execute the query and read into a DataFrame
    selected_division = selected_division.lower()
    select_query = text(f"SELECT * FROM division_goods_depowise_yearly_earning WHERE division_code = '{selected_division}'")
    earnings_df = pd.read_sql_query(select_query, conn)
    return earnings_df

def load_division_goods_monthwise_outward_data(selected_division):
    # Execute the query and read into a DataFrame
    selected_division = selected_division.lower()
    select_query = text(f"SELECT * FROM division_goods_monthwise_outward WHERE division_code = '{selected_division}'")
    earnings_df = pd.read_sql_query(select_query, conn)
    return earnings_df

# Define function to load division earnings data
def load_division_target_data(selected_division):
    # Execute the query and read into a DataFrame
    selected_division = selected_division.lower()
    select_query = text(f"SELECT * FROM division_target WHERE division_code = '{selected_division}'")
    earnings_df = pd.read_sql_query(select_query, conn)
    return earnings_df

def load_division_commoditywise_yearly_data(selected_division):
    selected_division = selected_division.lower()
    select_query = text(f"SELECT * FROM division_goods_commodity_outward WHERE division_code = '{selected_division}'")
    earnings_df = pd.read_sql_query(select_query, conn)
    return earnings_df

class load_goods_page:
    def __init__(self , selected_division):
        self.selected_division = selected_division
        self.division_goods_outward_df =load_division_goods_monthwise_outward_data(selected_division=self.selected_division)
        self.divisiion_target_df = load_division_target_data(selected_division=selected_division)
        self.division_depowise_outward_df = load_division_good_yearly_depowisesoutward_data(selected_division=selected_division)
        self.division_commodity_wise_outward_df = load_division_commoditywise_yearly_data(selected_division)
        
        goods_outward = self.division_goods_outward_df
        current_date = datetime.datetime.now()
        current_year = current_date.year
        current_month = current_date.month
        
        

        if current_month < 4:  # Financial year starts from April
            start_year = current_year - 1
            end_year = current_year
        else:
            start_year = current_year
            end_year = current_year + 1
            
        
        
        financial_year_start = pd.Timestamp(year=start_year, month=4, day=1)
        financial_year_end = pd.Timestamp(year=end_year, month=3, day=31)
        # march_year_end = pd.Timestamp(year=end_year, month=3, day=1)
       
        self.current_year_monthwise_outward_data = goods_outward[(goods_outward['earning_month'] >= financial_year_start.date()) &
                                        (goods_outward['earning_month'] <= financial_year_end.date())]
        
        previous_financial_year_start = pd.Timestamp(
            year=start_year-1, month=4, day=1)
        previous_financial_year_end = pd.Timestamp(
            year=end_year-1, month=3, day=31)
        self.previous_year_monthwise_outward_data = goods_outward[(goods_outward['earning_month'] >= previous_financial_year_start.date()) &
                                        (goods_outward['earning_month'] <= previous_financial_year_end.date())]
       
        current_year = pd.Timestamp(
            year=start_year+1, month=3, day=31)
        previous_year = pd.Timestamp(
            year=start_year, month=3, day=31)
        self.current_year_depowise_df = self.division_depowise_outward_df[(self.division_depowise_outward_df['earning_year'] == current_year.date())]
                      
        self.previous_year_depowise_df = self.division_depowise_outward_df[(self.division_depowise_outward_df['earning_year'] == previous_year.date())]
        
        self.current_year_commoditywise_ouward_df = self.division_commodity_wise_outward_df[(self.division_commodity_wise_outward_df['earning_year'] == current_year.date())]
        
        self.previous_year_commoditywise_outward_df = self.division_commodity_wise_outward_df[(self.division_commodity_wise_outward_df['earning_year'] == previous_year.date())]
    # def plot_monthwise_outward_comparison_plot(self):
    #     return 
        
        


selected_division = 'pune'
class_object = load_goods_page(selected_division)


In [98]:
class_object.previous_year_commoditywise_outward_df.head(30)

,id,earning_year,rakes,wagon,weight,freight,commodity,division_code
0,36,2023-03-31,345.0,14208.0,900544.0,901097.0,SUGAR,pune
1,37,2023-03-31,355.0,17674.0,915587.0,1093616.0,POL,pune
2,38,2023-03-31,0.0,0.0,0.0,0.0,DRY GRASS,pune
3,39,2023-03-31,603.0,15416.0,785217.0,800236.0,AUTOMOBILE,pune
4,40,2023-03-31,0.0,0.0,0.0,0.0,IRON ORE,pune
5,41,2023-03-31,0.0,0.0,0.0,0.0,PIG IRON,pune
6,42,2023-03-31,0.0,0.0,0.0,0.0,BENTONITE POWDER,pune
7,43,2023-03-31,0.0,0.0,0.0,0.0,CEMENT,pune
8,44,2023-03-31,1.0,42.0,2661.0,2704.0,FOODGRAIN,pune
9,45,2023-03-31,0.0,0.0,0.0,0.0,IRON STEEL,pune


In [ ]:
current_year_monthwise_outward_df = class_object.current_year_monthwise_outward_data
previous_year_monthwise_outward_df = class_object.previous_year_monthwise_outward_data
division_monthwise_target_df = class_object.divisiion_target_df


In [ ]:
division_monthwise_target_df.head(30)

,earning_id,earning_month,uts_sub_urban_pass,non_sub_pass,total_pass,freight,other_coaching,total,sundry,gross_total,sub_passengers,non_sub_passengers,total_passengers,division_code
0,1,2023-04-01,0.75,78.73,79.48,30.73,9.34,119.55,1.67,121.22,1.32,2.05,3.37,pune
1,2,2023-05-01,0.98,81.14,82.12,29.03,12.49,123.64,1.67,125.31,1.57,2.12,3.69,pune
2,3,2023-06-01,0.99,83.97,84.96,38.93,13.35,137.24,1.67,138.91,1.64,2.21,3.85,pune
3,4,2023-07-01,1.06,96.04,97.10,44.73,10.13,151.96,1.67,153.63,1.64,2.56,4.20,pune
4,5,2023-08-01,1.15,92.46,93.61,38.06,8.93,140.60,1.67,142.27,1.82,2.68,4.50,pune
5,6,2023-09-01,1.17,93.11,94.28,42.91,11.65,148.84,1.67,150.51,2.04,2.85,4.89,pune
6,7,2023-10-01,1.15,85.21,86.36,32.83,11.71,130.90,1.67,132.57,1.86,2.97,4.83,pune
7,8,2023-11-01,1.24,98.28,99.52,47.54,9.47,156.53,1.67,158.20,2.15,3.20,5.35,pune
8,9,2023-12-01,1.22,98.34,99.56,40.64,9.76,149.96,1.67,151.63,2.08,3.14,5.22,pune
9,10,2024-01-01,1.22,102.20,103.42,31.64,9.21,144.27,1.67,145.94,2.06,3.21,5.27,pune


In [ ]:
current_year_monthwise_outward_df = current_year_monthwise_outward_df.sort_values(by='earning_month')
previous_year_monthwise_outward_df = previous_year_monthwise_outward_df.sort_values(by='earning_month')
division_monthwise_target_df  = division_monthwise_target_df.sort_values(by='earning_month')
months = ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar']
# Create a figure with subplots
fig = go.Figure()

# Add trend lines for each category
fig.add_trace(
    go.Scatter(x=months,
               y=current_year_monthwise_outward_df['freight'],
               mode='lines+markers',
               showlegend=True,
               name='CY Freight',
               hovertemplate='Month: %{x}<br>Current Year Freight: %{y:.2f} Cr')
)
fig.add_trace(
    go.Scatter(x=months,
               y=previous_year_monthwise_outward_df['freight'],
               mode='lines+markers',
               showlegend=True,
               name='LY Freight',
               hovertemplate='Month: %{x}<br>Last Year Freight: %{y:.2f} Cr')
)

fig.add_trace(
    go.Scatter(x=months,
               y=division_monthwise_target_df['freight'],
               mode='lines+markers',
               showlegend=True,
               name='TGT Freight',
               hovertemplate='Month: %{x}<br>Last Year Freight: %{y:.2f} Cr')
    ),

# Update figure layout
fig.update_layout(
    xaxis_tickangle=-45,
    title='Revenue Trend Lines ',
    title_x=0.1,# Center title
    # xaxis_title='Month',
    yaxis_title='Revenue in Cr',
    margin=dict(t=10, b=10,r=10),  # Adjust top margin
    legend=dict(
        x=0.3,  # Change the x-coordinate of the legend
            y=1.15, # Change the y-coordinate of the legend
            traceorder='normal',  # Order of the legend items
            font=dict(
                family='sans-serif',
                size=12,
                color='black'
            ),
            bgcolor='rgba(255, 255, 255, 0.5)',  # Background color of the legend
            bordercolor='rgba(0, 0, 0, 0.5)',  # Border color of the legend
            borderwidth=1,  # Border width of the legend
            orientation='h',
            # yanchor="bottom",
            ),
        )
fig.update_xaxes(showline=True, linewidth=2, linecolor='black',)



In [ ]:

import plotly.graph_objects as go
def comparison_line_plot(feature , current_year_df , previous_year_df , target_df = None):
    months = ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar']
    # Create a figure with subplots
    fig = go.Figure()

    # Add trend lines for each category
    fig.add_trace(
        go.Scatter(x=months,
                y=current_year_df[feature],
                mode='lines+markers',
                showlegend=True,
                name='CY Freight',
                hovertemplate='Month: %{x}<br>Current Year Freight: %{y:.2f} Cr'),
        
    )
    fig.add_trace(
        go.Scatter(x=months,
                y=previous_year_df[feature],
                mode='lines+markers',
                showlegend=True,
                name='LY Freight',
                hovertemplate='Month: %{x}<br>Last Year Freight: %{y:.2f} Cr')
    )
    if target_df is not None:
        fig.add_trace(
            go.Scatter(x=months,
                    y=division_monthwise_target_df[feature],
                    mode='lines+markers',
                    showlegend=True,
                    name='TGT Freight',
                    hovertemplate='Month: %{x}<br>Last Year Freight: %{y:.2f} Cr')
        )

    # Annotate with monthly average revenue

    monthly_avg_current = current_year_df[feature].mean()
    monthly_avg_previous = previous_year_monthwise_outward_df[feature].mean()

    # Calculate daily average revenue assuming 30 days in a month
    daily_avg_current = monthly_avg_current / 30
    daily_avg_previous = monthly_avg_previous / 30
    fig.add_annotation(x=9,
                    y=15,
                    text=f'Monthly Avg CY: {monthly_avg_current:.2f} Cr',
                    showarrow=False,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#ff7f0e",
                    opacity=0.8,
                    font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="#ffffff"
                ),
                    )
    fig.add_annotation(x=9,
                        y=20,
                        text=f'Daily Avg CY: {daily_avg_current:.2f} Cr',
                        showarrow=False,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#ff7f0e",
                    opacity=0.8,
                    font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="#ffffff",
                        )
    )


    # Update figure layout
    fig.update_layout(
        xaxis_tickangle=-45,
        title='Revenue Trend Lines ',
        title_x=0.1,# Center title
        yaxis_title='Revenue in Cr',
        margin=dict(t=10, b=10,r=10),  # Adjust top margin
        legend=dict(
            x=0.3,  # Change the x-coordinate of the legend
            y=1.15, # Change the y-coordinate of the legend
            traceorder='normal',  # Order of the legend items
            font=dict(
                family='sans-serif',
                size=12,
                color='black'
            ),
            bgcolor='rgba(255, 255, 255, 0.5)',  # Background color of the legend
            bordercolor='rgba(0, 0, 0, 0.5)',  # Border color of the legend
            borderwidth=1,  # Border width of the legend
            orientation='h',
        ),
    )


    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',)

    # Show plot
    fig.show()
comparison_line_plot('freight', current_year_monthwise_outward_df , previous_year_monthwise_outward_df, target_df=division_monthwise_target_df)


In [ ]:
current_year_monthwise_outward_df.columns

Index(['id', 'earning_month', 'rakes', 'wagon', 'weight', 'freight',
       'division_code'],
      dtype='object')

In [ ]:
current_year_depowise_yearly_df = class_object.current_year_depowise_df
previous_year_depowose_yearly_df = class_object.previous_year_depowise_df
def yearly_data_comparison_line_plot(feature, current_year_df, previous_year_df, target_df=None):
    current_year_df = current_year_df.sort_values(by=feature)
    previous_year_df = previous_year_df.sort_values(by=feature)
    
    fig = go.Figure()
    
    if feature == 'freight':
        unit = 'Crores'
    elif feature == 'weight':
        unit = 'Tonnes'
    else:
        unit = ' '
    # Add trend lines for each category
    fig.add_trace(
        go.Scatter(x=current_year_df['station_code'],
                y=current_year_df[feature],
                mode='lines+markers',
                showlegend=True,
                name=f'CY : {feature[0].upper()}{feature[1:]}',
                hovertemplate = f'Station: %{{x}}<br>Current Year {feature[0].upper()}{feature[1:]}: %{{y:.2f}} {unit}'
        )
    )
    fig.add_trace(
        go.Scatter(x=current_year_df['station_code'],
                y=previous_year_df[feature],
                mode='lines+markers',
                showlegend=True,
                name=f'LY : {feature[0].upper()}{feature[1:]}',
                hovertemplate = f'Station: %{{x}}<br>Last Year {feature[0].upper()}{feature[1:]}: %{{y:.2f}} {unit}'
        )
    )
    
    # Update figure layout
    fig.update_layout(
        xaxis_tickangle=-45,
        title=f'Depowise {feature[0].upper()}{feature[1:]} Comparison',
        title_x=0.15,# Center title
        yaxis_title=f'{feature[0].upper()}{feature[1:]} {unit}',
        margin=dict(t=10, b=10,r=10),  # Adjust top margin
        legend=dict(
            x=0.50,  # Change the x-coordinate of the legend
            y=1.15, # Change the y-coordinate of the legend
            traceorder='normal',  # Order of the legend items
            font=dict(
                family='sans-serif',
                size=12,
                color='black'
            ),
            bgcolor='rgba(255, 255, 255, 0.5)',  # Background color of the legend
            bordercolor='rgba(0, 0, 0, 0.5)',  # Border color of the legend
            borderwidth=1,  # Border width of the legend
            orientation='h',
        ),
        yaxis=dict(
        title_font=dict(size=15),  # Adjust font size of y-axis title
    ),
    )


    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',)

    return fig.show()


yearly_data_comparison_line_plot('rakes', current_year_depowise_yearly_df , previous_year_depowose_yearly_df)

In [ ]:
class_object.current_year_commoditywise_ouward_df.columns

Index(['id', 'earning_year', 'rakes', 'wagon', 'weight', 'freight',
       'commodity', 'division_code'],
      dtype='object')

In [124]:
import plotly.express as px
df_filtered = class_object.current_year_commoditywise_ouward_df[class_object.current_year_commoditywise_ouward_df['freight'] > (class_object.current_year_commoditywise_ouward_df['freight']*0.001)]
df_filtered_ly = class_object.previous_year_commoditywise_outward_df[class_object.previous_year_commoditywise_outward_df['freight'] > (class_object.previous_year_commoditywise_outward_df['freight'].sum()*0.001)]


In [154]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_pie_chart(current_year_commoditywise_ouward_df,previous_year_commoditywise_outward_df):
    df_filtered = current_year_commoditywise_ouward_df[class_object.current_year_commoditywise_ouward_df['freight'] > (class_object.current_year_commoditywise_ouward_df['freight']*0.001)]
    df_filtered_ly = previous_year_commoditywise_outward_df[class_object.previous_year_commoditywise_outward_df['freight'] > (class_object.previous_year_commoditywise_outward_df['freight'].sum()*0.001)]

    fig = make_subplots(rows=2, cols=1, specs=[[{'type':'domain'}], [{'type':'domain'}]])

    # Extract values and names from the filtered DataFrame
    values = df_filtered['freight']
    names = df_filtered['commodity']
    values_ly = df_filtered_ly['freight']
    names_ly = df_filtered_ly['commodity']
    # Add pie charts to the subplots
    fig.add_trace(go.Pie(values=values, labels=names, title=f'CY : Frieght Distribution<br>by Commodity ',textinfo='label+percent', showlegend=False,insidetextorientation='radial'), row=1, col=1)
    fig.add_trace(go.Pie(values=values_ly, labels=names_ly, title=f'LY : Freight Distribution<br>by Commodity',textinfo='label+percent', showlegend=False,insidetextorientation='radial'), row=2, col=1)
    fig.update_traces(hole=.8, hoverinfo="label+percent+name")
    fig.update_layout(
        margin=dict(t=10, b=10,r=10),
        legend=dict(
            x=0.01,  # Change the x-coordinate of the legend
            y=1.15, # Change the y-coordinate of the legend
            traceorder='normal',  # Order of the legend items
            font=dict(
                family='sans-serif',
                size=10,
                color='black'
            ),
            bgcolor='rgba(255, 255, 255, 0.5)',  # Background color of the legend
            bordercolor='rgba(0, 0, 0, 0.5)',  # Border color of the legend
            borderwidth=1,  # Border width of the legend
            orientation='v',
        ),
    )
    return fig.show()
plot_pie_chart(class_object.current_year_commoditywise_ouward_df, class_object.previous_year_commoditywise_outward_df)